In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup,NavigableString, Tag
import time
import requests
import re
import csv
from selenium.webdriver.common.by import By
import pandas as pd
import os

headers = {'content-type': 'text/html; charset=UTF-8','user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
}
#s = Service(r"/usr/local/bin/chromedriver") #使用chromedriver、selenium
driver = webdriver.Chrome()
    
driver.get("https://www.cardu.com.tw/") #自動打開網頁（不要關）
    
time.sleep(5) #暫停
    
soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
#print(soup.prettify())  #輸出排版後的HTML內容

#爬卡排行的名稱
card_ranking_type = [] 
results1 = soup.find_all('div', class_='text-center')
for results in results1:
    result = results.find('a').get('title')
    card_ranking_type.append(result)
    #print(result) #測試抓到的卡排行
    
card_ranking_type.pop(0) #頭兩個不需要 因此刪除
card_ranking_type.pop(0)
#for i in range(len(card_ranking_type)):
    #print(card_ranking_type[i])

#爬卡排行的網址
card_ranking_link = []
results1 = soup.find_all('div', class_='text-center')
for results in results1:
    result = results.find('a').get('href')
    result = 'https://www.cardu.com.tw/'+result
    card_ranking_link.append(result)
    #print(result)

card_ranking_link.pop(0) #頭兩個不需要 因此刪除
card_ranking_link.pop(0)
#for i in range(len(card_ranking_link)):
    #print(card_ranking_link[i])

ranking_dict = {
    '排行名稱':card_ranking_type,
    '排行連結':card_ranking_link,
}
df = pd.DataFrame(ranking_dict)
folder_name = 'carducsv'
path = "/Users/takoyaki/Downloads/專題爬蟲" 
folder_path = os.path.join(path, folder_name)
if not os.path.isdir(folder_path):
    os.makedirs(folder_path, mode=0o777)
path0 = str(path+'/'+folder_name+"/")
#path='/Users/takoyaki/Downloads/專題爬蟲/carducsv/'
df.to_csv(path0+'icardu排行.csv', index = False)


for k in range(len(card_ranking_link)):
    
    card_name1 = [] #卡名無門檻
    card_name2 = [] #卡名有門檻
    card_name = [] #卡名一般排行
    card_link1 = [] #卡連結無門檻
    card_link2 = [] #卡連結有門檻
    card_link = [] #卡連結一般排行
    #一般
    block_name = []
    block_one = [] #國內回饋
    block_two = [] #國外回饋
    block_three = [] #其他卡片
    #無回饋
    block_one1 = [] #國內回饋
    block_two1 = [] #國外回饋
    block_three1 = [] #其他卡片
    #有回饋
    block_one2 = [] #國內回饋
    block_two2 = [] #國外回饋
    block_three2 = [] #其他卡片
    
    print('☆'+card_ranking_type[k])
    driver.get(card_ranking_link[k]) #自動打開網頁（不要關）
    time.sleep(3) #暫停   
    soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
    
    results = soup.find_all('div', class_='col-md text-center col-4')
    for result in results:
        print(result.text)
        block_name.append(result.text)
    
    if card_ranking_type[k] == '現金回饋':
        
        button= driver.find_element(By.ID, "goodSet-tab") #打開有門檻
        button.click()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
        
        i=0
        results = soup.find_all('div', class_='col-md-11 wx-100-ph')
        for result in results:
            name = result.find('h5', class_='money_main mb-0') #卡名
            link = result.find('a').get('href') #卡連結
            
            if i<10: #無門檻
                card_name1.append(name.text)
                card_link1.append('https://www.cardu.com.tw'+link[2:])
            elif i==10: #有門檻換行
                print('\n')
                card_name2.append(name.text)
                card_link2.append('https://www.cardu.com.tw'+link[2:])
            elif i>10: #有門檻
                card_name2.append(name.text)
                card_link2.append('https://www.cardu.com.tw'+link[2:])
            print(name.text)
            print('https://www.cardu.com.tw'+link[2:])
            i+=1
        results=soup.find_all('div', class_='row w-h-100') #幾張卡
        
        j=0
        for result in results:
            if j < 10: #無門檻
                print(j)
                i = 0
                result1 = result.find_all('div', class_='col-md hv-center phone_block col-4 py-2 pc_rank_txt') #國內回饋+其他卡片
                for re in result1:
                    if i == 0:
                        print(block_name[0] +':'+ re.find('p').text)
                        block_one1.append(re.find('p').text) #國內回饋
                    else:
                        print(block_name[2] +':'+ re.find('p').text)
                        block_three1.append(re.find('p').text) #其他卡片
                    i+=1
                    
                result2 = result.find('div',class_='col-md hv-center border-left border-right phone_block col-4 py-2 pc_rank_txt') #國外回饋
                print(block_name[1] +':'+ result2.find('p').text)
                block_two1.append(result2.find('p').text)
                
            elif j == 10: #換行
                print(j)
                i = 0
                result1 = result.find_all('div', class_='col-md col-4 hv-center phone_block pc_rank_txt') #國內回饋+其他卡片
                for re in result1:
                    if i == 0:
                        print(block_name[0] +':'+ re.find('p').text)
                        block_one2.append(re.find('p').text) #國內回饋
                    else:
                        print(block_name[2] +':'+ re.find('p').text)
                        block_three2.append(re.find('p').text) #其他卡片 
                    i+=1
                    
                result2 = result.find('div',class_='col-md col-4 hv-center border-left border-right phone_block pc_rank_txt') #國外回饋
                print(block_name[1] +':'+ result2.find('p').text)
                block_two2.append(result2.find('p').text)
                print('\n')
                
            elif j > 10: #有門檻
                print(j)
                i = 0
                result1 = result.find_all('div', class_='col-md col-4 hv-center phone_block pc_rank_txt') #國內回饋+其他卡片
                for re in result1:
                    if i == 0:
                        print(block_name[0] +':'+ re.find('p').text)
                        block_one2.append(re.find('p').text) #國內回饋
                    else:
                        print(block_name[2] +':'+ re.find('p').text)
                        block_three2.append(re.find('p').text) #其他卡片
                    i+=1
                    
                result2 = result.find('div',class_='col-md col-4 hv-center border-left border-right phone_block pc_rank_txt') #國外回饋
                print(block_name[1] +':'+ result2.find('p').text)
                block_two2.append(result2.find('p').text)
                
            j+=1  
            
        ranking_card_dict1 = { #無門檻
            '卡名':card_name1,
            #'卡連結':card_link1,
            block_name[0]:block_one1,
            block_name[1]:block_two1,
            block_name[2]:block_three1
        }
        ranking_card_dict2 = { #有門檻
            '卡名':card_name2,
            #'卡連結':card_link2,
            block_name[0]:block_one2,
            block_name[1]:block_two2,
            block_name[2]:block_three2
        }
        
        df1 = pd.DataFrame(ranking_card_dict1)
        df2 = pd.DataFrame(ranking_card_dict2)
        folder_name = 'carducsv'
        path = "/Users/takoyaki/Downloads/專題爬蟲"
        #path='/Users/takoyaki/Downloads/專題爬蟲/carducsv/'
        folder_path = os.path.join(path, folder_name)
        if not os.path.isdir(folder_path):
            os.makedirs(folder_path, mode=0o777)
        path0 = str(path+'/'+folder_name+"/")
        df1.to_csv(path0+'cardu'+card_ranking_type[k]+'無門檻排行.csv', index = False)
        df2.to_csv(path0+'cardu'+card_ranking_type[k]+'有門檻排行.csv', index = False)
        print('\n')
    else:
        results = soup.find_all('div', class_='col-md-11 wx-100-ph')
        for result in results:
            name = result.find('h5', class_='money_main mb-0')
            link = result.find('a').get('href')
            print(name.text)
            card_name.append(name.text)
            card_link.append('https://www.cardu.com.tw'+link[2:])
            print('https://www.cardu.com.tw'+link[2:])
            
        results=soup.find_all('div', class_='row w-h-100')
        for result in results:
            result1 = result.find_all('div', class_='col-md hv-center phone_block col-4 py-2 pc_rank_txt') #國內回饋+其他卡片
            i = 0
            for re in result1:
                if i == 0:
                    print(block_name[0] +':'+ re.find('p').text)
                    block_one.append(re.find('p').text) #國內回饋
                else:
                    print(block_name[2] +':'+ re.find('p').text)
                    block_three.append(re.find('p').text) #其他卡片
                i+=1
                
            result2 = result.find('div',class_='col-md hv-center border-left border-right phone_block col-4 py-2 pc_rank_txt') #國外回饋
            print(block_name[1] +':'+ result2.find('p').text)
            block_two.append(result2.find('p').text)
            
        ranking_card_dict = { #一般
            '卡名':card_name,
            #'卡連結':card_link,
            block_name[0]:block_one,
            block_name[1]:block_two,
            block_name[2]:block_three
        }
        df = pd.DataFrame(ranking_card_dict)
        
        folder_name = 'carducsv'
        path = "/Users/takoyaki/Downloads/專題爬蟲"  

        folder_path = os.path.join(path, folder_name)
        if not os.path.isdir(folder_path):
            os.makedirs(folder_path, mode=0o777)
        path0 = str(path+'/'+folder_name+"/")
        #path='/Users/takoyaki/Downloads/專題爬蟲/carducsv/'
        df.to_csv(path0+'cardu'+card_ranking_type[k]+'排行.csv', index = False)
        print('\n')
    
    if card_ranking_type[k] == '現金回饋':
        for i in range(len(card_link1)+len(card_link2)):
            if i<10: #無門檻
                driver.get(card_link1[i]) #自動打開網頁（不要關）
                print(card_name1[i])
                name = card_name1[i]
            else: #有門檻
                driver.get(card_link2[i-10]) #自動打開網頁（不要關）
                print(card_name2[i-10])
                name = card_name2[i-10]
            time.sleep(3) #暫停
            soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
            
            #卡圖片
            card_img = []
            result = soup.find('img', class_='ccard_img')
            if result !=None:
                card_img.append(result.get('src'))
                print(result.get('src'))
                
                #申請連結
                card_apply_link = []
                result = soup.find('a', class_='btn btn-block warning-layered btnOver')
                if result != None:
                    card_apply_link.append(result.get('href'))
                    print(result.get('href'))
                else:
                    card_apply_link.append('無')
                           
                #優惠
                summary_title = []
                summary_text = []
                results = soup.find_all('div', class_='col-md-6 py-1')
                for result in results:
                    print(result.text)
                    summary_text.append(result.text[:-1])
                    summary_title.append('特色')
                           
                #全部權益           
                button= driver.find_element(By.ID, "special_3-tab") 
                button.click()
                time.sleep(2)  
                soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
                           
                #上標題
                title = []
                result = soup.find('form', class_='credit_boot')
                result1 = result.find_all('th')
                for result in result1:
                    print(result.text[:4])
                    title.append(result.text[:4])
                    
                #內文
                title_list = [] #標題
                text_list = [] #內容
                #result2 = soup.find('table')
                #print(result2)
                result3 = soup.find('div', id ='special_3')
                result2 = result3.find('tbody')
                #print(result2)
                result2 = result2.find_all('tr')
                for result in result2:
                    i = 0
                    results = result.find_all('td')
                    for result in results:
                        if i == 0:
                            print(result.find('label').text)
                            title_list.append(result.find('label').text)
                        else:
                            print(result.text)
                            text_list.append(result.text)
                        i+=1
                keyword='、'.join(summary_text) #把大標當關鍵字
    
                card_dict = { 
                    #'卡圖片':card_img,
                    #'申請連結':card_apply_link,
                    summary_title[0]:keyword
                }
                
                card_dict1={
                    title[0]:title_list,
                    title[1]:text_list
                }
                
                df1 = pd.DataFrame(card_dict1)
                df = pd.DataFrame(card_dict)
                folder_name = 'carducsv'+card_ranking_type[k]
                path = "/Users/takoyaki/Downloads/專題爬蟲"  
        
                folder_path = os.path.join(path, folder_name)
                if not os.path.isdir(folder_path):
                    os.makedirs(folder_path, mode=0o777)
                path0 = str(path+'/'+folder_name+"/")
                #path='/Users/takoyaki/Downloads/專題爬蟲/carducsv/'
                df1.to_csv(path0+'cardu'+name+'權益.csv', index = False)
                df.to_csv(path0+'cardu'+name+'.csv', index = False)
                
            else:
                cards = soup.find_all('a', class_='card_name my-2')
                for card in cards:
                    card_img = []
                    driver.get('https://www.cardu.com.tw'+card.get('href')[2:]) #自動打開網頁（不要關）
                    time.sleep(3) #暫停
                    soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
                    
                    result = soup.find('img', class_='ccard_img')
                    card_img.append(result.get('src'))
                    print(result.get('src'))
                    
                    #申請連結
                    card_apply_link = []
                    result = soup.find('a', class_='btn btn-block warning-layered btnOver')
                    if result != None:
                        card_apply_link.append(result.get('href'))
                        print(result.get('href'))
                    else:
                        card_apply_link.append('無')
                               
                    #優惠
                    summary_title = []
                    summary_text = []
                    results = soup.find_all('div', class_='col-md-6 py-1')
                    for result in results:
                        print(result.text)
                        summary_text.append(result.text[:-1])
                        summary_title.append('特色')
                               
                    #全部權益           
                    button= driver.find_element(By.ID, "special_3-tab") 
                    button.click()
                    time.sleep(2)  
                    soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
                               
                    #上標題
                    title = []
                    result = soup.find('form', class_='credit_boot')
                    result1 = result.find_all('th')
                    for result in result1:
                        print(result.text[:4])
                        title.append(result.text[:4])
                        
                    #內文
                    title_list = [] #標題
                    text_list = [] #內容
                    #result2 = soup.find('table')
                    #print(result2)
                    result3 = soup.find('div', id ='special_3')
                    result2 = result3.find('tbody')
                    #print(result2)
                    result2 = result2.find_all('tr')
                    for result in result2:
                        i = 0
                        results = result.find_all('td')
                        for result in results:
                            if i == 0:
                                print(result.find('label').text)
                                title_list.append(result.find('label').text)
                            else:
                                print(result.text)
                                text_list.append(result.text)
                            i+=1
                    keyword='、'.join(summary_text) #把大標當關鍵字
        
                    card_dict = { 
                        #'卡圖片':card_img,
                        #'申請連結':card_apply_link,
                        summary_title[0]:keyword
                    }
                    
                    card_dict1={
                        title[0]:title_list,
                        title[1]:text_list
                    }
                    
                    df1 = pd.DataFrame(card_dict1)
                    df = pd.DataFrame(card_dict)
                    
                    folder_name = 'carducsv'+card_ranking_type[k]
                    path = "/Users/takoyaki/Downloads/專題爬蟲"  
            
                    folder_path = os.path.join(path, folder_name)
                    if not os.path.isdir(folder_path):
                        os.makedirs(folder_path, mode=0o777)
                    path0 = str(path+'/'+folder_name+"/")
                    
                    #path='/Users/takoyaki/Downloads/專題爬蟲/carducsv/'
                    df1.to_csv(path0+'cardu'+card_name[0]+'權益.csv', index = False)
                    df.to_csv(path0+'cardu'+card_name[0]+'.csv', index = False) 
            
    else:
        for i in range(len(card_link)):
            driver.get(card_link[i]) #自動打開網頁（不要關）
            print(card_name[i])
            time.sleep(3) #暫停
            soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
            
            #卡圖片
            card_img = []
            result = soup.find('img', class_='ccard_img')
            if result !=None:
                card_img.append(result.get('src'))
                print(result.get('src'))
                
                #申請連結
                card_apply_link = []
                result = soup.find('a', class_='btn btn-block warning-layered btnOver')
                if result != None:
                    card_apply_link.append(result.get('href'))
                    print(result.get('href'))
                else:
                    card_apply_link.append('無')
                           
                #優惠
                summary_title = []
                summary_text = []
                results = soup.find_all('div', class_='col-md-6 py-1')
                for result in results:
                    print(result.text)
                    summary_text.append(result.text[:-1])
                    summary_title.append('特色')
                           
                #全部權益           
                button= driver.find_element(By.ID, "special_3-tab") 
                button.click()
                time.sleep(2)  
                soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
                           
                #上標題
                title = []
                result = soup.find('form', class_='credit_boot')
                result1 = result.find_all('th')
                for result in result1:
                    print(result.text[:4])
                    title.append(result.text[:4])
                    
                #內文
                title_list = [] #標題
                text_list = [] #內容
                #result2 = soup.find('table')
                #print(result2)
                result3 = soup.find('div', id ='special_3')
                result2 = result3.find('tbody')
                #print(result2)
                result2 = result2.find_all('tr')
                for result in result2:
                    i = 0
                    results = result.find_all('td')
                    for result in results:
                        if i == 0:
                            print(result.find('label').text)
                            title_list.append(result.find('label').text)
                        else:
                            print(result.text)
                            text_list.append(result.text)
                        i+=1
                keyword='、'.join(summary_text) #把大標當關鍵字
    
                card_dict = { 
                    #'卡圖片':card_img,
                    #'申請連結':card_apply_link,
                    summary_title[0]:keyword
                }
                
                card_dict1={
                    title[0]:title_list,
                    title[1]:text_list
                }
                
                df1 = pd.DataFrame(card_dict1)
                df = pd.DataFrame(card_dict)
                folder_name = 'carducsv'+card_ranking_type[k]
                path = "/Users/takoyaki/Downloads/專題爬蟲"  
        
                folder_path = os.path.join(path, folder_name)
                if not os.path.isdir(folder_path):
                    os.makedirs(folder_path, mode=0o777)
                path0 = str(path+'/'+folder_name+"/")
                #path='/Users/takoyaki/Downloads/專題爬蟲/carducsv/'
                
                df1.to_csv(path0+'icardu'+card_name[0]+'權益.csv', index = False)
                df.to_csv(path0+'icardu'+card_name[0]+'.csv', index = False)
                
            else:
                cards = soup.find_all('a', class_='card_name my-2')
                for card in cards:
                    card_img = []
                    driver.get('https://www.cardu.com.tw'+card.get('href')[2:]) #自動打開網頁（不要關）
                    time.sleep(3) #暫停
                    soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
                    
                    result = soup.find('img', class_='ccard_img')
                    card_img.append(result.get('src'))
                    print(result.get('src'))
                    
                    #申請連結
                    card_apply_link = []
                    result = soup.find('a', class_='btn btn-block warning-layered btnOver')
                    if result != None:
                        card_apply_link.append(result.get('href'))
                        print(result.get('href'))
                    else:
                        card_apply_link.append('無')
                               
                    #優惠
                    summary_title = []
                    summary_text = []
                    results = soup.find_all('div', class_='col-md-6 py-1')
                    for result in results:
                        print(result.text)
                        summary_text.append(result.text[:-1])
                        summary_title.append('特色')
                               
                    #全部權益           
                    button= driver.find_element(By.ID, "special_3-tab") 
                    button.click()
                    time.sleep(2)  
                    soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
                               
                    #上標題
                    title = []
                    result = soup.find('form', class_='credit_boot')
                    result1 = result.find_all('th')
                    for result in result1:
                        print(result.text[:4])
                        title.append(result.text[:4])
                        
                    #內文
                    title_list = [] #標題
                    text_list = [] #內容
                    #result2 = soup.find('table')
                    #print(result2)
                    result3 = soup.find('div', id ='special_3')
                    result2 = result3.find('tbody')
                    #print(result2)
                    result2 = result2.find_all('tr')
                    for result in result2:
                        i = 0
                        results = result.find_all('td')
                        for result in results:
                            if i == 0:
                                print(result.find('label').text)
                                title_list.append(result.find('label').text)
                            else:
                                print(result.text)
                                text_list.append(result.text)
                            i+=1
                    keyword='、'.join(summary_text) #把大標當關鍵字
        
                    card_dict = { 
                        #'卡圖片':card_img,
                        #'申請連結':card_apply_link,
                        summary_title[0]:keyword
                    }
                    
                    card_dict1={
                        title[0]:title_list,
                        title[1]:text_list
                    }
                    
                    df1 = pd.DataFrame(card_dict1)
                    df = pd.DataFrame(card_dict)
                    folder_name = 'carducsv'+card_ranking_type[k]
                    path = "/Users/takoyaki/Downloads/專題爬蟲"  
            
                    folder_path = os.path.join(path, folder_name)
                    if not os.path.isdir(folder_path):
                        os.makedirs(folder_path, mode=0o777)
                    path0 = str(path+'/'+folder_name+"/")
                    #path='/Users/takoyaki/Downloads/專題爬蟲/carducsv/'
                    
                    df1.to_csv(path0+'icardu'+card_name[0]+'權益.csv', index = False)
                    df.to_csv(path0+'icardu'+card_name[0]+'.csv', index = False)
